In [ ]:
import torch
import torchvision
import Dataset as D
import transform as T
from torch.utils.data import DataLoader
from validation import validation
from plot import show_hist

In [ ]:
lr = 0.001
batch_size = 128

In [ ]:
# dataset
img_labels = D.ImageLabels('/mnt/training_labels.txt')
dataset, val_dataset = D.get_train_val_dataset('/mnt/training_images', img_labels=img_labels,
                                               transform=T.get_transform(), val=0.15)
class_transformer = D.ClassTransform('/mnt/classes.txt')
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=32)

In [ ]:
device = torch.device('cuda:0')

In [ ]:
# model
model = torch.nn.Sequential(
    torchvision.models.resnet50(pretrained=True),
    torch.nn.Linear(1000, class_transformer.total_size),
)
model = torch.nn.DataParallel(model)
model.to(device)

In [ ]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-6)

In [ ]:
# loss
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
# training history
train_loss, validation_loss = [], []
train_acc, validation_acc = [], []

In [ ]:
from IPython.display import clear_output

# training loop
epoch = 100
for i in range(epoch):
    # train
    model.train()
    loss_sum, positive, cnt = 0., 0, 0
    for labels, images in data_loader:
        y_true = torch.tensor(class_transformer.to_order(labels)).to(device)
        y_pred = model(images.to(device))

        loss = loss_function(y_pred, y_true)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # metrics
        cnt += 1
        loss_sum += loss.item()
        pre_class = class_transformer.to_class_name(y_pred)
        for ind, pre_label in enumerate(pre_class):
            if pre_class == labels[ind]:
                positive += 1

    # validation
    model.eval()
    metrics = validation(model, val_dataset, class_transformer, loss_function, device)

    # record the loss
    train_loss.append(loss_sum/cnt)
    validation_loss.append(metrics['loss'])

    # record the acc
    train_acc.append(positive / len(dataset))
    validation_acc.append(metrics['acc'])

    print('epoch', i, 'finish')
    print(metrics)

    
    clear_output()
    show_hist([train_loss, validation_loss], 'loss', ['train', 'validation'])
    show_hist([train_acc, validation_acc], 'acc', ['train', 'validation'])